# NYC Parking Tickets: Violation Rate Prediction

In [111]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

from datetime import datetime, timedelta

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# importing libraries

import csv
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [112]:
selected_columns = [
    'Street Code1',
    'Street Code2',
    'Street Code3',
    'Street Name',
    # 'Violation Location',
    'Violation Post Code',
    'Intersecting Street',
    'Violation Precinct',
    'Violation County',
    'Vehicle Body Type',
    'Issue Date',
    # 'Date First Observed',
    'Violation Time',
    #'Days Parking In Effect',
    #'From Hours In Effect',
    #'To Hours In Effect',
    'Violation Code',
]

'''
Genel olarak, "county" (ilçe) daha büyük bir alanı temsil eder. Bir "county", genellikle belirli bir eyaletin sınırları içinde bulunan büyük bir coğrafi bölgeyi kapsar. Bir eyalette birden çok "county" bulunabilir ve her biri kendi yerel yönetimine sahiptir.

Öte yandan, "precinct" (seçim bölgesi) daha küçük bir coğrafi bölgeyi ifade eder. Bu, genellikle seçmenlerin oy kullandığı belirli bir mahalle veya köyü temsil eder. Seçim bölgesi, seçimlerde kullanılan ve oy verme işlemini düzenleyen bir kavramdır.
'''

n_rows_to_read = 3000000
file_path = 'Parking_Violations_2022.csv'
df = pd.read_csv(file_path, usecols=selected_columns, nrows=n_rows_to_read) # nrows=n_rows_to_read
df

,Issue Date,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code
0,06/25/2021,40,VAN,63430,69230,13490,88,0130A,K,NORTH PORTLAND AVE,NaN,NaN
1,06/25/2021,20,SUBN,13490,40404,40404,88,0225A,K,AUBURN PLACE,NaN,NaN
2,06/17/2021,98,SDN,79430,47130,11750,88,0809P,K,SOUTH PORTLAND AVE,NaN,NaN
3,06/16/2021,98,SDN,53130,23230,23930,83,0605P,K,JEFFERSON ST,NaN,NaN
4,07/04/2021,40,TAXI,81030,23930,40030,83,1058P,K,STANHOPE ST,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,09/16/2021,36,4DSD,0,0,0,0,0231P,QN,WB LINDEN BLVD @ AMB,ER ST,NaN
2999996,09/16/2021,36,SUBN,0,0,0,0,0231P,QN,SB QUEENS BLVD @ 87T,H AVE,NaN
2999997,09/16/2021,36,SUBN,0,0,0,0,0231P,BK,WB MEEKER AVE @ LORI,MER ST,NaN
2999998,09/16/2021,36,SUBN,0,0,0,0,0231P,BX,NB BRONX RIVER AVE @,E 174TH ST,NaN


In [113]:
df['Violation Precinct'].isna().sum()

0

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 12 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Issue Date           object
 1   Violation Code       int64 
 2   Vehicle Body Type    object
 3   Street Code1         int64 
 4   Street Code2         int64 
 5   Street Code3         int64 
 6   Violation Precinct   int64 
 7   Violation Time       object
 8   Violation County     object
 9   Street Name          object
 10  Intersecting Street  object
 11  Violation Post Code  object
dtypes: int64(5), object(7)
memory usage: 274.7+ MB


In [115]:
df['Violation Time']

0          0130A
1          0225A
2          0809P
3          0605P
4          1058P
           ...  
2999995    0231P
2999996    0231P
2999997    0231P
2999998    0231P
2999999    0231P
Name: Violation Time, Length: 3000000, dtype: object

In [116]:
df_1 = df.copy()

In [117]:
# import sweetviz as sv
# analyze_report = sv.analyze(df_1)
# analyze_report.show_html('analyze.html', open_browser=False)

In [118]:
df_1['Violation Date'] = pd.to_datetime(df_1['Issue Date'])
df_1['Violation Date (Month)'] = df_1['Violation Date'].dt.month
df_1['Violation Date (Day)'] = df_1['Violation Date'].dt.day
df_1['Violation Date (Year)'] = df_1['Violation Date'].dt.year
df_1 = df_1.drop(columns=['Issue Date'])

In [119]:
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
0,40,VAN,63430,69230,13490,88,0130A,K,NORTH PORTLAND AVE,NaN,NaN,2021-06-25,6,25,2021
1,20,SUBN,13490,40404,40404,88,0225A,K,AUBURN PLACE,NaN,NaN,2021-06-25,6,25,2021
2,98,SDN,79430,47130,11750,88,0809P,K,SOUTH PORTLAND AVE,NaN,NaN,2021-06-17,6,17,2021
3,98,SDN,53130,23230,23930,83,0605P,K,JEFFERSON ST,NaN,NaN,2021-06-16,6,16,2021
4,40,TAXI,81030,23930,40030,83,1058P,K,STANHOPE ST,NaN,NaN,2021-07-04,7,4,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,36,4DSD,0,0,0,0,0231P,QN,WB LINDEN BLVD @ AMB,ER ST,NaN,2021-09-16,9,16,2021
2999996,36,SUBN,0,0,0,0,0231P,QN,SB QUEENS BLVD @ 87T,H AVE,NaN,2021-09-16,9,16,2021
2999997,36,SUBN,0,0,0,0,0231P,BK,WB MEEKER AVE @ LORI,MER ST,NaN,2021-09-16,9,16,2021
2999998,36,SUBN,0,0,0,0,0231P,BX,NB BRONX RIVER AVE @,E 174TH ST,NaN,2021-09-16,9,16,2021


In [120]:
def convert_to_24hr_format(time_str):
    try:
        if '.' in time_str:
            return None
        if time_str[-1] == 'P':
            return '{:02d}:{:02d}'.format((int(time_str[:2]) + 12) % 24, int(time_str[2:4]))
        elif time_str[-1] == 'A':
            return '{:02d}:{:02d}'.format(int(time_str[:2]) % 12, int(time_str[2:4]))
        else:
            return None
    except (ValueError, TypeError):
        return None

df_1['Violation Time'] = df_1['Violation Time'].astype(str)
df_1.loc[:, 'Violation Time'] = df_1['Violation Time'].apply(convert_to_24hr_format)
df_1 = df_1.dropna(subset=['Violation Time'])

In [121]:
df_1.loc[:, 'Violation Time'] = pd.to_datetime(df_1['Violation Time'], format='%H:%M', errors='coerce').dt.time

In [122]:
df_1['Violation Time']

0          01:30:00
1          02:25:00
2          20:09:00
3          18:05:00
4          22:58:00
             ...   
2999995    14:31:00
2999996    14:31:00
2999997    14:31:00
2999998    14:31:00
2999999    14:31:00
Name: Violation Time, Length: 2999953, dtype: object

In [123]:
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
0,40,VAN,63430,69230,13490,88,01:30:00,K,NORTH PORTLAND AVE,NaN,NaN,2021-06-25,6,25,2021
1,20,SUBN,13490,40404,40404,88,02:25:00,K,AUBURN PLACE,NaN,NaN,2021-06-25,6,25,2021
2,98,SDN,79430,47130,11750,88,20:09:00,K,SOUTH PORTLAND AVE,NaN,NaN,2021-06-17,6,17,2021
3,98,SDN,53130,23230,23930,83,18:05:00,K,JEFFERSON ST,NaN,NaN,2021-06-16,6,16,2021
4,40,TAXI,81030,23930,40030,83,22:58:00,K,STANHOPE ST,NaN,NaN,2021-07-04,7,4,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,36,4DSD,0,0,0,0,14:31:00,QN,WB LINDEN BLVD @ AMB,ER ST,NaN,2021-09-16,9,16,2021
2999996,36,SUBN,0,0,0,0,14:31:00,QN,SB QUEENS BLVD @ 87T,H AVE,NaN,2021-09-16,9,16,2021
2999997,36,SUBN,0,0,0,0,14:31:00,BK,WB MEEKER AVE @ LORI,MER ST,NaN,2021-09-16,9,16,2021
2999998,36,SUBN,0,0,0,0,14:31:00,BX,NB BRONX RIVER AVE @,E 174TH ST,NaN,2021-09-16,9,16,2021


In [124]:
df_1.sort_values(by=['Violation Date', 'Violation Time'], inplace=True)
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
6647,20,SUBN,72720,27180,29220,46,00:04:00,BX,WALTON AVE,NaN,NaN,2000-06-27,6,27,2000
977151,20,SDN,0,40404,40404,71,19:40:00,K,WINTHROP,NaN,NaN,2000-06-27,6,27,2000
3082,78,NaN,0,40404,40404,105,22:00:00,Q,BROOKVILL,NaN,NaN,2000-06-28,6,28,2000
5337,35,VAN,10610,40404,40404,14,00:52:00,NY,7 AVE,NaN,NaN,2000-07-15,7,15,2000
1370970,51,VAN,13610,24490,32595,26,20:43:00,NY,BROADWAY,NaN,NaN,2000-07-22,7,22,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372288,98,SDN,20090,17390,51090,102,20:43:00,Q,107 ST,NaN,NaN,2031-07-29,7,29,2031
2827701,98,SDN,36620,26730,26760,44,17:31:00,BX,GRANT AVENUE,NaN,NaN,2031-08-24,8,24,2031
1786231,21,SUBN,28890,16290,16590,102,08:32:00,Q,ATLANTIC AVE,NaN,NaN,2032-08-06,8,6,2032
2819824,14,SDN,38470,31250,15575,61,08:43:00,K,EATON COURT,NaN,NaN,2061-08-27,8,27,2061


In [125]:
start_date = pd.to_datetime('2021-07-01')
end_date = pd.to_datetime('2022-06-30')

In [126]:
df_1 = df_1[(df_1['Violation Date'] >= start_date) & (df_1['Violation Date'] <= end_date)]
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
19777,21,4DSD,0,0,0,107,00:00:00,Qns,Opposite of 70-87 PA,RK DRIVE EAST,NaN,2021-07-01,7,1,2021
22002,0,VAN,0,0,0,0,00:00:00,NaN,NaN,NaN,NaN,2021-07-01,7,1,2021
23604,0,SUBN,0,0,0,0,00:00:00,NaN,NaN,NaN,NaN,2021-07-01,7,1,2021
26384,21,4DSD,8120,25860,25890,40,00:00:00,Bronx,ALEXANDER AVENUE,NaN,NaN,2021-07-01,7,1,2021
28018,21,SUBN,0,0,0,78,00:00:00,Kings,Front Of 190 12 STRE,ET,NaN,2021-07-01,7,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964965,40,SDN,49820,52220,61220,49,02:25:00,BX,MATTHEWS AVE,NaN,NaN,2022-06-10,6,10,2022
975313,21,SUBN,42730,27530,53630,79,08:22:00,K,FULTON ST,NaN,NaN,2022-06-21,6,21,2022
963609,98,SDN,38920,74830,76550,52,04:13:00,BX,HEATH AVE,NaN,NaN,2022-06-23,6,23,2022
990848,17,P-U,11115,12670,26630,1,11:34:00,NY,WEST STREET,NaN,NaN,2022-06-23,6,23,2022


In [127]:
df_1['Violation Hour'] = df_1['Violation Time'].apply(lambda x: (x.hour % 24)).astype(int)
df_1['Violation Hour']

19777      0
22002      0
23604      0
26384      0
28018      0
          ..
964965     2
975313     8
963609     4
990848    11
971803     4
Name: Violation Hour, Length: 2748287, dtype: int32

In [129]:
def calculate_location_violation_rate(df, location, time_range, precinct, violation_date):
    start_time = datetime.strptime(time_range[0], "%H:%M").time()
    end_time = datetime.strptime(time_range[1], "%H:%M").time()

    # filtered_data = df[(df['Street Name'] == location) & (df['Violation Date'] == violation_date) & (~df['Violation Time'].isna()) &
    #                    (df['Violation Time'].apply(lambda x: not pd.isna(x) and x == x and start_time <= x <= end_time))]

    filtered_data = df[(df['Street Name'] == location) &
                   (df['Violation Date'] == violation_date) &
                   (~df['Violation Time'].isna()) &
                   (df['Violation Time'].apply(lambda x: start_time <= x <= end_time if pd.notna(x) else False))]

    print("filtered data:", len(filtered_data))

    total_location_violations = len(df_1[(df_1['Violation Time'].between(start_time, end_time)) &
                                      (df_1['Violation Precinct'] == precinct) &
                                      (df_1['Violation Date'] == violation_date)])

    print("total_location_violations:", total_location_violations)

    if total_location_violations == 0:
        return 0

    location_violations = len(filtered_data)

    location_violation_rate = (location_violations / total_location_violations) * 100

    return location_violation_rate

location = 'GRANDVIEW AVE'
location_precinct = 121
time_range = ('00:00', '23:00')
violation_date = pd.to_datetime('2021-07-25')

rate = calculate_location_violation_rate(df_1, location, time_range, location_precinct, violation_date)
print(f'Location Violation Rate for {location} between {time_range[0]} and {time_range[1]}: {rate:.2f}%')

filtered data: 7
total_location_violations: 18
Location Violation Rate for GRANDVIEW AVE between 00:00 and 23:00: 38.89%


In [ ]:
import dask.dataframe as dd
from dask.distributed import Client
from datetime import datetime, timedelta

ddf = dd.from_pandas(df_1, npartitions=10)

client = Client()

def calculate_violation_rate(row):
    location = row['Street Name']
    hour = row['Violation Hour']
    violation_date = row['Violation Date']
    precinct = row['Violation Precinct']

    if hour == 23:
        start_time = datetime.strptime('23:00', '%H:%M').time()
        end_time = datetime.strptime('23:59', '%H:%M').time()
    else:
        start_time = datetime.strptime(f'{hour:02d}:00', '%H:%M').time()
        end_time = (datetime.strptime(f'{hour+1:02d}:00', '%H:%M') + timedelta(minutes=59, seconds=59)).time()

    total_location_violations = len(ddf[(ddf['Violation Time'].between(start_time, end_time)) &
                                        (ddf['Violation Precinct'] == precinct) &
                                        (ddf['Violation Date'] == violation_date)])

    location_violations = len(ddf[(ddf['Street Name'] == location) &
                                   (ddf['Violation Time'].between(start_time, end_time)) &
                                   (ddf['Violation Date'] == violation_date)])

    if total_location_violations == 0:
        return 0

    location_violation_rate = (location_violations / total_location_violations) * 100

    return location_violation_rate

ddf['Violation Rate'] = ddf.apply(calculate_violation_rate, axis=1, meta=('x', 'f8'))

result_with_violation_rate = ddf.compute()

client.close()
result_with_violation_rate

In [130]:
def calculate_violation_rate(row):
    location = row['Street Name']
    hour = row['Violation Hour']
    violation_date = row['Violation Date']
    precinct = row['Violation Precinct']

    # Handle the case where hour is 24
    if hour == 23:
        start_time = datetime.strptime('23:00', '%H:%M').time()
        end_time = datetime.strptime('23:59', '%H:%M').time()
    else:
        start_time = datetime.strptime(f'{hour:02d}:00', '%H:%M').time()
        end_time = (datetime.strptime(f'{hour+1:02d}:00', '%H:%M') + timedelta(minutes=59, seconds=59)).time()


    total_location_violations = len(df_1[(df_1['Violation Time'].between(start_time, end_time)) &
                                      (df_1['Violation Precinct'] == precinct) &
                                      (df_1['Violation Date'] == violation_date)])

    location_violations = len(df_1[(df_1['Street Name'] == location) &
                                 (df_1['Violation Time'].between(start_time, end_time)) &
                                 (df_1['Violation Date'] == violation_date)])

    if total_location_violations == 0:
        return 0

    location_violation_rate = (location_violations / total_location_violations) * 100

    return location_violation_rate

if 'Violation Rate' in df_1:
    df_1.drop('Violation Rate', axis=1, inplace=True)

df_1['Violation Rate'] = df_1.apply(lambda row: calculate_violation_rate(row), axis=1)

KeyboardInterrupt: 

In [ ]:
df_1['Violation Rate']

In [ ]:
df_1

In [ ]:
def scale_between_0_and_1(value, min_value, max_value):
    """
    Scale a value to be between 0 and 1.

    :param value: The value to be scaled.
    :param min_value: The minimum value in the range of your data.
    :param max_value: The maximum value in the range of your data.
    :return: Scaled value between 0 and 1.
    """
    if max_value == min_value:
        return 0  # Avoid division by zero if all values are the same

    return (value - min_value) / (max_value - min_value)

In [ ]:
df_1['Violation Rate (Scaled)'] = df_1['Violation Rate'].apply(lambda x: scale_between_0_and_1(x, df_1['Violation Rate'].min(), df_1['Violation Rate'].max()))
df_1['Violation Rate (Scaled)']

In [ ]:
df_1['Violation Rate'].max()